<h1><center>Final Capstone Project.</center></h1>

<h3><center>ALY6140 80956 Analytics Systems Technology SEC 04 Spring 2021 CPS</center></h3>
<h2><center><b>Customer Churn Modeling.</b></center></h2>



Team Members:
1) Krishna Naga Mohan Gollapalli
1) Neil Mascarenhas

#### Date: 10 May 2021

# Importing Libraries

### Basics and required.

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")  #### Hides warning messages to get cleaner outputs.
sns.set_theme(style = 'whitegrid')

In [6]:
import os
import sys 

sys.path.insert(0, os.path.abspath('.'))  ## Setting the path to current Directory


import Capstone_Group01_Utility as utility  ## Importing the external utility file. So we can use it whenever requied.

utility.Test()  ## Testing if the package is imported sucessfully and we can execute the definations.

Capstone Test Successful


### Required for modeling

In [7]:
### In here we import all the packages and libraries for modeing. 
### We are primarilay using Sklearn but if needed we will explore more 


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

## For model selection and cross validation.
from sklearn.metrics import accuracy_score,confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score,f1_score,classification_report,roc_curve,plot_roc_curve,auc,precision_recall_curve,plot_precision_recall_curve,average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

### Ussed for encoding and feature engineering. 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder